In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
import shutil

# Define the directories for images and labels
base_dir = '/kaggle/input/dataset-cvat/images/default'
images_dir = '/kaggle/working/images'
labels_dir = '/kaggle/working/labels'

# Create directories if they do not exist
os.makedirs(images_dir, exist_ok=True)
os.makedirs(labels_dir, exist_ok=True)

# Copy images to the images directory and labels to the labels directory
for filename in os.listdir(base_dir):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        shutil.copy(os.path.join(base_dir, filename), os.path.join(images_dir, filename))
    elif filename.endswith('.txt'):
        shutil.copy(os.path.join(base_dir, filename), os.path.join(labels_dir, filename))


In [3]:
from sklearn.model_selection import train_test_split
import numpy as np

# List all images
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
# Corresponding label files
label_files = [f'{os.path.splitext(f)[0]}.txt' for f in image_files]

# Split the dataset into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    image_files, label_files, test_size=0.2, random_state=42)

# Function to move files
def move_files(files, source_dir, target_dir):
    for f in files:
        shutil.move(os.path.join(source_dir, f), os.path.join(target_dir, f))

# Create directories for the train and validation splits
train_images_dir = '/kaggle/working/train/images'
train_labels_dir = '/kaggle/working/train/labels'
val_images_dir = '/kaggle/working/val/images'
val_labels_dir = '/kaggle/working/val/labels'

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Move the files
move_files(train_images, images_dir, train_images_dir)
move_files(train_labels, labels_dir, train_labels_dir)
move_files(val_images, images_dir, val_images_dir)
move_files(val_labels, labels_dir, val_labels_dir)


In [4]:
%cd /kaggle/working/
!mkdir tmp
%cd tmp

/kaggle/working
/kaggle/working/tmp


In [5]:
# Install YOLOv5 dependencies from the official repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!pip install -r requirements.txt  # install dependencies


Cloning into 'yolov5'...
remote: Enumerating objects: 16582, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 16582 (delta 30), reused 39 (delta 18), pack-reused 16522
Receiving objects: 100% (16582/16582), 15.13 MiB | 30.92 MiB/s, done.
Resolving deltas: 100% (11387/11387), done.
/kaggle/working/tmp/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 56.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.4/756.4 kB 40.5 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spopt 0.6.0 requires shapely>=2.0.1, but you have 

In [6]:
import torch
from IPython.display import Image, clear_output  # to display images
import numpy as np
import xml.etree.ElementTree as ET
from os.path import join
import os
import cv2

In [7]:
def replace_extension(filename):
    # This will handle both .png and .jpg files and replace them with .txt
    base, ext = os.path.splitext(filename)
    if ext.lower() in ['.png', '.jpg', '.jpeg']:
        return base + '.txt'
    return filename


def parse_xml_to_dict(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    annotations = []
    for image in root.findall('image'):
        width = int(image.get('width'))
        height = int(image.get('height'))
        filename = image.get('name')

        for poly in image.findall('.//polygon'):
            points = poly.get('points').split(';')
            x_coords = []
            y_coords = []
            for point in points:
                x, y = map(float, point.split(','))
                x_coords.append(x)
                y_coords.append(y)

            # Calculate the bounding box
            xmin = min(x_coords)
            xmax = max(x_coords)
            ymin = min(y_coords)
            ymax = max(y_coords)

            # Convert to YOLO format
            x_center = ((xmin + xmax) / 2) / width
            y_center = ((ymin + ymax) / 2) / height
            box_width = (xmax - xmin) / width
            box_height = (ymax - ymin) / height

            label = poly.get('label').lower()  # Assuming class labels are consistent and correctly mapped
            annotations.append((filename, label, x_center, y_center, box_width, box_height))

    return annotations

def save_annotations_to_txt(annotations, output_dir, class_dict):
    for annotation in annotations:
        filename, label, x_center, y_center, box_width, box_height = annotation
        label_idx = class_dict[label]
        yolo_format = f"{label_idx} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n"
        
        txt_filename = replace_extension(filename)
        txt_path = join(output_dir, txt_filename)

        with open(txt_path, 'a') as file:
            file.write(yolo_format)

# Define your class labels and corresponding indices
class_dict = {'building': 0, 'damage': 1, 'roof': 2, 'damaged roof': 3, 'broken window': 4, 'other': 5}

In [8]:
# Set paths
xml_file_path = '/kaggle/input/dataset-cvat/annotations.xml'
img_dir = '/kaggle/input/dataset-cvat/images/default'
output_dir = 'yolo_format'
# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Parse XML file
annotations = parse_xml_to_dict(xml_file_path)

# Save annotations in YOLO format
save_annotations_to_txt(annotations, output_dir, class_dict)



In [9]:
import yaml


In [10]:
%cp -r /kaggle/working/tmp

cp: missing destination file operand after '/kaggle/working/tmp'
Try 'cp --help' for more information.


In [11]:
training_dir = '/kaggle/working/train'
validation_dir = '/kaggle/working/val' 
data_yaml = dict(
    train = training_dir,
    val = validation_dir,
    nc = 6,  # Number of classes
    names = ['building', 'damage', 'roof', 'damaged roof', 'broken window', 'other']  # Class names
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)


In [ ]:
import os
os.environ['WANDB_MODE'] = 'disabled'  # Disable WandB
!python train.py --img 640 --batch 16 --epochs 50 --data data.yaml --weights yolov5s.pt --cache

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-05-11 14:30:54.001893: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-11 14:30:54.001994: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered


In [ ]:
# Load the trained model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='runs/train/exp/weights/best.pt')

In [ ]:
# Use in case of inference on a single image
# Set the path to an image
img_path = '/kaggle/working/val/images/download10_2.png'

# Inference
results = model(img_path)
results.print()  # Print results

# Display the image with bounding boxes
results.show()  # Display the image with bounding boxes

In [ ]:
# Same inference as above
# !python detect.py --weights runs/train/exp/weights/best.pt --conf 0.5 --img-size 640 --source /kaggle/working/val/images/image7_1.png